In [2]:
import json
import logging
from sqlalchemy import Column, Integer, String, Float, DateTime, Boolean, func
from iotfunctions import bif
from custom.functions import InvokeModel
from iotfunctions.metadata import EntityType
from iotfunctions.db import Database
from iotfunctions.base import BaseTransformer
from iotfunctions.bif import EntityDataGenerator
#from iotfunctions.enginelog import EngineLogging
from custom import settings
import datetime as dt

import pandas as pd
import numpy as np

from watson_machine_learning_client import WatsonMachineLearningAPIClient
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.pipeline import Pipeline
import sklearn


/Users/kkbankol@us.ibm.com/projects/watson-analytics-dashboard/env/lib/python3.7/site-packages/iotfunctions/bif.py:1945: UserWarning: IoTCalcSettings is deprecated. Use entity type constants instead of a metadata provider to set entity type properties
  warnings.warn(('IoTCalcSettings is deprecated. Use entity type constants'

2020-03-31 15:54:40,518 - root - INFO - 
Logging Set To: DEBUG


Loading WML Environment Variables
Environment Variables Loaded Successfully


In [12]:
# set Monitor credentials here
credentials = "" #

SyntaxError: invalid syntax (<ipython-input-12-a503e9a08966>, line 2)

In [4]:
db = Database(credentials = credentials)
db_schema = None #  set if you are not using the default

## create and register entity
if settings.ENTITY_NAME:
    entity_name = settings.ENTITY_NAME

print("creating entity: " + entity_name)

entity = EntityType(entity_name, db,
                    Column('deviceid',String(50)),
                    Column('anomaly_score', Integer()),
                    Column("torque", Integer()),
                    Column("acc", Integer()),
                    Column("load", Integer()),
                    Column("tool_type", Integer()),
                    Column("speed", Float()),
                    Column("travel_time", Float()),
                    **{
                      '_timestamp' : 'evt_timestamp',
                      # '_production_mode': False,
                      '_db_schema' : db_schema}
                   )
entity.register(raise_error=False)
print("Entity registered")


2020-03-31 15:54:43,180 - iotfunctions.db - INFO - 
Connection string for SqlAlchemy => db2): db2+ibm_db://iotanalytics:I0TAnalytics2@2@@dashdb-enterprise-yp-dal12-125.services.dal.bluemix.net:50000/BLUDB;

2020-03-31 15:54:43,180 - iotfunctions.db - INFO - 
Connection string for SqlAlchemy => db2): db2+ibm_db://iotanalytics:I0TAnalytics2@2@@dashdb-enterprise-yp-dal12-125.services.dal.bluemix.net:50000/BLUDB;

2020-03-31 15:54:43,183 - iotfunctions.db - DEBUG - 
created a CosClient object

2020-03-31 15:54:44,094 - iotfunctions.db - DEBUG - 
Db connection established

2020-03-31 15:54:59,756 - iotfunctions.db - DEBUG - 
http request successful. status 200


In [ ]:
db.register_functions([InvokeModel])
print("Function registered")

In [6]:
## generate sample data for entity
print("Generating sample data")
entity.generate_data(days=2.0, drop_existing=True)


## add a column to store output, set as zeroes
df = db.read_table(table_name=entity_name, schema=db_schema)
empty_columns = ["anomaly_score"]
print("Setting " +  ", ".join(empty_columns) + "columns to zeros ")
df['anomaly_score'] = np.zeros(len(df))
db.write_frame(df, table_name=entity_name, schema=db_schema, if_exists="replace")
db.commit()
print("DataFrame written to db")


Generating sample data



2020-03-31 15:55:08,293 - iotfunctions.db - DEBUG - 
Dropped table name KB_ANOMALY

2020-03-31 15:55:09,028 - iotfunctions.db - DEBUG - 
Recreated table KB_ANOMALY

2020-03-31 15:55:09,029 - iotfunctions.metadata - INFO - 
dropped tables []

2020-03-31 15:55:09,029 - iotfunctions.metadata - INFO - 
dropped tables []

2020-03-31 15:55:09,031 - iotfunctions.metadata - DEBUG - 
Generating data for KB_ANOMALY with metrics ['torque', 'tool_type', 'acc', 'speed', 'load', 'anomaly_score', 'travel_time'] and dimensions ['eventtype'] and dates []

2020-03-31 15:55:09,076 - iotfunctions.automation - DEBUG - 
Generated 2886 rows of time series data from 2020-03-29 22:50:09.032851 to 2020-03-31 22:55:09.032851
/Users/kkbankol@us.ibm.com/projects/watson-analytics-dashboard/env/lib/python3.7/site-packages/pandas/io/sql.py:1336: UserWarning: The provided table name 'KB_ANOMALY' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider usi

Setting anomaly_scorecolumns to zeros 



2020-03-31 15:55:17,973 - iotfunctions.db - INFO - 
Wrote data to table kb_anomaly 

2020-03-31 15:55:17,973 - iotfunctions.db - INFO - 
Wrote data to table kb_anomaly 


DataFrame written to db


In [7]:
## register a model


# reduce entity dataframe to relevant input columns
if settings.INPUT_COLUMNS:
    columns = (settings.INPUT_COLUMNS).split(', ')
else:
    columns = []

if len(columns) > 0:
    print("filtering columns")
    print(columns)
    s_df = df[columns]
else:
    print("using original dataframe")
    s_df = df

print(s_df.head())

# convert dataframe to 2d array (as expected by WML)
rows = [list(r) for i,r in s_df.iterrows()]

# create WML client
wml_creds_path = '/Users/kkbankol@us.ibm.com/projects/maximo_anomaly/wml_credentials.json'
wml_credentials = json.loads(open(wml_creds_path).read())
client = WatsonMachineLearningAPIClient( wml_credentials )

# create a model (using IsolationForest here with default settings)
clf = IsolationForest()

# place classifier in a sklearn "pipeline"
pipeline = Pipeline( steps = [ ( "classifier", clf ) ] )

# fit data to model pipeline
pipeline.fit(rows)

# test model
pipeline.predict(rows)

# publish pipeline to WML
sk_version = sklearn.__version__
metadata = {
    client.repository.ModelMetaNames.FRAMEWORK_VERSION: 'scikit-learn',
    client.repository.ModelMetaNames.FRAMEWORK_VERSION: sk_version,
    client.repository.ModelMetaNames.NAME: 'anomaly_model',
    client.repository.ModelMetaNames.RUNTIME_NAME: 'python',
    client.repository.ModelMetaNames.RUNTIME_VERSION: '3.6'
}
model_details_inmem = client.repository.store_model( pipeline, meta_props=metadata)
model_id_inmem = model_details_inmem["metadata"]["guid"]
deployment_details_inmem = client.deployments.create( artifact_uid=model_id_inmem, name="anomaly_model" )
deployment_id = deployment_details_inmem["metadata"]["guid"]
model_endpoint_url_inmem = client.deployments.get_scoring_url( deployment_details_inmem )
print("model invocation endpoint: " + model_endpoint_url_inmem)
print("Place below line in .env file like so")
print("WATSON_ML_DEPLOYMENT_ID=" + deployment_id) # TODO, need to confirm notebook will reload .env file. Or just change from .env to mem


filtering columns
['torque', 'acc', 'load', 'speed', 'tool_type', 'travel_time']
   torque  acc  load     speed  tool_type  travel_time
0       1    0     0  1.202717          0    -2.014539
1       0    0     0  1.919618          0     1.250091
2       0    0     2  0.371211          0     0.537916
3       0    1     1  0.605652          0    -0.632042
4       0    0     0  0.366713          0     0.618659


/Users/kkbankol@us.ibm.com/projects/watson-analytics-dashboard/env/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)

2020-03-31 15:58:13,803 - ModelCollection - DEBUG - 
Creating a new scikit pipeline model: anomaly_model

2020-03-31 15:58:15,999 - watson_machine_learning_client.libs.repo.swagger_client.rest - DEBUG - 
response body: {"metadata":{"guid":"ec8a3916-865f-4725-858c-f1845f3a17ef","url":"https://us-south.ml.cloud.ibm.com/v3/ml_assets/models/ec8a3916-865f-4725-858c-f1845f3a17ef","created_at":"2020-03-31T22:58:14.215Z"},"entity":{"name":"anomaly_model","model_url":"/v3/ml_assets/models/ec8a3916-865f-4725-858c-f1845f3a17ef","framework":{"name":"scikit-learn","version":"0.20","runtimes":[{"name":"python","version":"3.6"}]},"model_version":{"url":"https://us-south.ml.cloud.ibm.com/v3/ml_assets/models/ec8a3916-865f-4725-858c-f1845f3a



#######################################################################################

Synchronous deployment creation for uid: 'ec8a3916-865f-4725-858c-f1845f3a17ef' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='86a53131-4a83-4199-bc66-f3e19d1ec471'
------------------------------------------------------------------------------------------------


model invocation endpoint: https://us-south.ml.cloud.ibm.com/v3/wml_instances/da1a8e1e-84bd-483d-bf4d-14f43b3b7a51/deployments/86a53131-4a83-4199-bc66-f3e19d1ec471/online
Place below line in .env file like so
WATSON_ML_DEPLOYMENT_ID=86a53131-4a83-4199-bc66-f3e19d1ec471


In [11]:
WATSON_ML_ENDPOINT = settings.WATSON_ML_ENDPOINT
WATSON_ML_INSTANCE_ID = settings.WATSON_ML_INSTANCE_ID
WATSON_ML_APIKEY = settings.WATSON_ML_APIKEY
WATSON_ML_DEPLOYMENT_ID = settings.WATSON_ML_DEPLOYMENT_ID

IAM_UID = settings.IAM_UID
IAM_PASSWORD = settings.IAM_PASSWORD

INPUT_COLUMNS = settings.INPUT_COLUMNS or []
if INPUT_COLUMNS and (len(INPUT_COLUMNS) > 0):
    INPUT_COLUMNS = INPUT_COLUMNS.replace(' ', '').split(',')
else:
    INPUT_COLUMNS = []

entity = EntityType(entity_name, db,
                # following columns can be dynamically generated based on meters associated with each asset
                Column('deviceid' ,String(50)),
                # Column('evt_timestamp',String(50)),
                Column('anomaly_score', Integer()),
                Column("torque", Integer()),
                Column("acc", Integer()),
                Column("load", Integer()),
                Column("tool_type", Integer()),
                Column("speed", Float()),
                Column("travel_time", Float()),
                InvokeModel(
                                wml_endpoint=WATSON_ML_ENDPOINT,
                                instance_id=WATSON_ML_INSTANCE_ID,
                                deployment_id=WATSON_ML_DEPLOYMENT_ID,
                                apikey=WATSON_ML_APIKEY,
                                input_columns=INPUT_COLUMNS,
                                output_item = 'anomaly_score_done'),
                **{
                  '_timestamp' : 'evt_timestamp',
                  '_db_schema' : db_schema}
)
entity.exec_local_pipeline()



2020-03-31 16:50:51,997 - root - DEBUG - 
body {}

2020-03-31 16:50:52,000 - root - DEBUG - 
column_map {}

2020-03-31 16:50:52,001 - iotfunctions.metadata - DEBUG - 
Initializing new entity type using iotfunctions 2.0.3

2020-03-31 16:50:52,002 - iotfunctions.util - DEBUG - 
Starting trace

2020-03-31 16:50:52,003 - iotfunctions.util - DEBUG - 
Trace name: auto_trace_kb_anomaly_20200331235052

2020-03-31 16:50:52,004 - iotfunctions.util - DEBUG - 
auto_save None

2020-03-31 16:50:52,005 - iotfunctions.metadata - WARNING - 
No _db_schema specified in **kwargs. Usingdefault database schema.

2020-03-31 16:50:52,005 - iotfunctions.metadata - WARNING - 
No _db_schema specified in **kwargs. Usingdefault database schema.

2020-03-31 16:50:52,006 - iotfunctions.util - DEBUG - 
categorizing arguments

2020-03-31 16:50:52,010 - iotfunctions.metadata - DEBUG - 
Initialized entity type 
EntityType:KB_ANOMALY
Functions:
InvokeModel at granularity None required inputs not evaluated yet outputs pr


2020-03-31 16:50:53,388 - root - DEBUG - 
Index(['deviceid', 'evt_timestamp', 'devicetype', 'logicalinterface_id',
       'eventtype', 'format', 'updated_utc', 'load', 'anomaly_score',
       'tool_type', 'torque', 'speed', 'travel_time', 'acc'],
      dtype='object')

2020-03-31 16:50:53,391 - root - DEBUG - 
posting enitity data to WML model

2020-03-31 16:50:54,231 - root - DEBUG - 
token successfully generated

2020-03-31 16:50:54,232 - root - DEBUG - 
posting to WML


filtering columns
Index(['deviceid', 'evt_timestamp', 'devicetype', 'logicalinterface_id',
       'eventtype', 'format', 'updated_utc', 'load', 'anomaly_score',
       'tool_type', 'torque', 'speed', 'travel_time', 'acc'],
      dtype='object')
Index(['torque', 'acc', 'load', 'speed', 'tool_type', 'travel_time'], dtype='object')



2020-03-31 16:50:54,717 - root - DEBUG - 
model response code: 200

2020-03-31 16:50:54,718 - root - DEBUG - 
model response

2020-03-31 16:50:54,720 - root - DEBUG - 
{"fields": ["prediction"], "values": [[1], [1], [1], [-1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [-1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [-1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [-1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1]]}


2020-03-31 16:50:54,721 - root - DEBUG - 
json

2020-03-31 16:50:54,722 - root - DEBUG - 
{'fields': ['prediction'], 'values': [[1], [1], [1], [-1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [-1], [1], [1], [1], [1], [1], [


2020-03-31 16:50:58,822 - iotfunctions.pipeline - INFO - 
Trace payload: {'produces_output_items': True, 'output_items': ['anomaly_score_done'], 'discard_prior_data': False, 'merge_result': 'existing empty df with new DataFrame', 'usage': 0, 'can_proceed': True, 'updated': datetime.datetime(2020, 3, 31, 23, 50, 58, 820981), 'cumulative_usage': 0}

2020-03-31 16:50:58,822 - iotfunctions.pipeline - INFO - 
Trace payload: {'produces_output_items': True, 'output_items': ['anomaly_score_done'], 'discard_prior_data': False, 'merge_result': 'existing empty df with new DataFrame', 'usage': 0, 'can_proceed': True, 'updated': datetime.datetime(2020, 3, 31, 23, 50, 58, 820981), 'cumulative_usage': 0}

2020-03-31 16:50:58,824 - iotfunctions.pipeline - DEBUG - 
Preload stages complete

2020-03-31 16:50:58,826 - iotfunctions.pipeline - DEBUG - 
Building a job spec for schedule 5min with subsumbed schedules ['5min']

2020-03-31 16:50:58,827 - iotfunctions.pipeline - DEBUG - 
Gathered stages of type 

ConnectionError: HTTPSConnectionPool(host='undefined', port=443): Max retries exceeded with url: /analytics-runtime-monitor-demo-395729cb6a06/Monitor-Demo/kb_anomaly/20200331/kb_anomaly_trace_235052 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x12d051cd0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))